In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 라이브러리와 app 데이터 세트 로딩

import numpy as np
import pandas as pd
import time
import gc #가비지 컬렉션 --> 메모리 관리

#for plotting
import matplotlib.pyplot as plt
import seaborn as sns

#import warning
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#화면의 크기 설정
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',200)

In [ ]:
#listdir를 사용해서 dir리스트 확인
print(os.listdir("../input/home-credit-default-risk"))

In [ ]:
def show_hist_by_target(df, columns):
    cond_1 = (df['TARGET'] == 1)
    cond_0 = (df['TARGET'] == 0)
    
    for column in columns:
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 4), squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=axs[0][0] )
        sns.distplot(df[cond_0][column], ax=axs[0][1], label='0', color='blue')
        sns.distplot(df[cond_1][column], ax=axs[0][1], label='1', color='red')   

In [ ]:
def nulldata(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms= ms[ms["Percent"] > 0]
    f,ax =plt.subplots(figsize=(15,10))
    plt.xticks(rotation='90')
    fig=sns.barplot(ms.index, ms["Percent"],color="green",alpha=0.8)
    plt.xlabel('Features', fontsize=15)
    plt.ylabel('Percent of null values', fontsize=15)
    plt.title('Percent null data by feature', fontsize=15)
    return ms

> **< EDA >**
- 데이터에서 어떠한 특징을 추출하기 전에 데이터를 잘 살펴봐야함
- 데이터의 분피 및 값을 검토함으로써 데이터를 관찰 및 이해를 통해 잠재적인 문제를 발견 가능함
- 다양한 각도에서 살펴보는 과정에서 다양한 패턴을 발견할 수 있고, 이를 통해 문제 해결하기위한 가설을 세울 수 있음

In [ ]:
#application_train.csv 파일 feature 확인

app_train = pd.read_csv("../input/home-credit-default-risk/application_train.csv")
print('Training data shape : ', app_train.shape)
app_train.head()

In [ ]:
#application_test.csv 파일 feature 확인

app_test = pd.read_csv("../input/home-credit-default-risk/application_test.csv")
print('Testing data shape : ', app_test.shape)
app_test.head()

In [ ]:
#training 데이터의 컬럼 확인  --> 122개
app_train.columns.values 


In [ ]:
#target의 1과 0의 비율 확인 --> 1은 상환이 어려움 0은 상환가능
app_train['TARGET'].value_counts()

In [ ]:
# 0과 1의 비율이 불균형함
# 따라서 추후에 Over sampling, Under sampling을 고려해야한다
app_train['TARGET'].astype(int).plot.hist()

In [ ]:
#target값에 따른 feature들이 어떤 특성을가지는지 EDA 분석

#1. CODE_GENDER
sns.catplot(x = 'CODE_GENDER', col='TARGET', data = app_train, kind = 'count')
# target 0에서는 여성의 비율이 높다

In [ ]:
#2. NAME_FAMILY_STATUS
chart = sns.catplot(x = 'NAME_FAMILY_STATUS', col='TARGET', data = app_train, kind = 'count')
chart.set_xticklabels(rotation=65)
# target 0의 경우  Married가 많고, target 1의 경우도 MArried가 많다

In [ ]:
#3. CNT_CHILDREN 
sns.catplot(x = 'CNT_CHILDREN', col='TARGET', data = app_train, kind = 'count')
# 별 특별한 소득 X

In [ ]:
#4. CNT_FAM_MEMBERS
sns.catplot(x = 'CNT_FAM_MEMBERS', col='TARGET', data = app_train, kind = 'count')
# 별 특별한 소득 X

In [ ]:
#5. NAME_INCOME_TYPE : 소득 유령을 나타냄
chart = sns.catplot(x = 'NAME_INCOME_TYPE', col='TARGET', data = app_train, kind = 'count')
chart.set_xticklabels(rotation=65)
# 일을 하는 사람이 상환하기 더 쉽다는것은 당연히 볼 수 있다

In [ ]:
#6. NAME_EDUCATION_TYPE : 교육을 받은 정도
chart = sns.catplot(x = 'NAME_EDUCATION_TYPE', col='TARGET', data = app_train, kind = 'count')
chart.set_xticklabels(rotation=65)
#2차교육, 고학력자일수록 대출을 더 많이하고 acdemic defreeㄴ느 매우 적다

In [ ]:
#7. NAME_HOUSING_TYPE = 거주지
chart = sns.catplot(x = 'NAME_HOUSING_TYPE', col='TARGET', data = app_train, kind = 'count')
chart.set_xticklabels(rotation=65)
# House/Apartment에 사는사람이 압도적으로 많다

In [ ]:
#8. AMT_INCOME_TOTAL : 소득수준
app_train['AMT_INCOME_TOTAL'].hist()
# 대부분의 값이 몰려있는것을 확인할 수 있다.

In [ ]:
#8-1. AMT_INCOME_TOAL의 데이터 살펴보기
app_train['AMT_INCOME_TOTAL'].head(15)
# 100000을기준으로 나누어보자

In [ ]:
#8-2. AMT_INCOME_TOTAL
app_train[app_train['AMT_INCOME_TOTAL'] < 1000000]['AMT_INCOME_TOTAL'].hist()
# 0.5까지가 유의미한 정보 인걸을 확인할 수 있음 
#500000이전까지의 정보를 더 세세하게 시각화할 필요 있음

In [ ]:
#8.2 AMT_INCOME_TOTAL
cond1 = (app_train['TARGET'] == 1)
cond0 = (app_train['TARGET'] == 0)
cond_amt = (app_train['AMT_INCOME_TOTAL'] < 500000)

sns.displot(app_train[cond0 & cond_amt]['AMT_INCOME_TOTAL'], label='0', color='blue')
sns.displot(app_train[cond1 & cond_amt]['AMT_INCOME_TOTAL'], label='0', color='red')
#두 데이터의 형태가 거의 비슷하다 유의한 정보는 없는듯하다

In [ ]:
#9. AMT_CREDIT = Credit amount of the loan 로 대출금액
app_train['AMT_CREDIT'].hist()

In [ ]:
#9-1. AMT_CREDIT = Credit amount of the loan 로 대출금액
#target1을 기준으로
cond1 = (app_train['TARGET'] == 1)
sns.displot(app_train[cond1]['AMT_CREDIT'])

In [ ]:
#9-2. AMT_CREDIT 의 데이터 형태확인
app_train['AMT_CREDIT'].head(15)

In [ ]:
#9-1. AMT_CREDIT의 2000000을 기준으로 히스토그램 확인 
# 사람들이 얼마를 대출하는지에 대한 분포확인
app_train[app_train['AMT_CREDIT'] < 2000000]['AMT_CREDIT'].hist()
#

In [ ]:
#10. DAYS_BIRTH 
columns = ['DAYS_BIRTH']
show_hist_by_target(app_train, columns)
# -10000정도를 보면 빨간부분(target1)의 비율이 높은것을 알 수 있음

In [ ]:
#10-1. DAYS_BIRTH와 TARGET변수와의 관계를 더 자세히 보면
plt.figure(figsize=(10,8))

# 제 때 대출을 상환하는 고객의 나이 plot (TARGET=0)
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0')

# 제 때 대출을 상환하지 못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')

plt.xlabel('Age(years)')
plt.xlabel('Density')
plt.title('Distribution of Ages')

# 확실히 젋은 사람이 대출상환을 못하고 있는 것을 볼 수 있다.

In [ ]:
# object 컬럼을 target에따라 count비교
object_columns = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()
object_columns

In [ ]:
def show_count_by_target(df, columns):
    cond_1 = (df['TARGET'] == 1)
    cond_0 = (df['TARGET'] == 0)
    
    for column in columns:
        fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)
        sns.countplot(df[cond_0][column], ax=ax[0][0]) # 제 때 대출을 상환(TARGET=0)
        sns.countplot(df[cond_1][column], ax=ax[0][1]) # 제 때 대출을 상환X(TARGET=1)

show_count_by_target(app_train, object_columns)

< Feature Engineering>
 - 데이터에 대한 지식을 활용하여 Target 예측모델의 정확도를 향상
 - Target을 분류할 수 있는 피처를 만드는 과정으로 다양한 컬럼(변수)결합, 상관관계, 피처 변환 등
     ex1. 컬럼1과 컬럼2,3 groupby를 통해 얻는 새로운 피처
     ex2. 대출 금액에 대하 관련 피처 생성, aggregation을 통한 피처 변환 등

In [ ]:
#위의 object type의 데이터를 통해 시각화 해본 결과 유의미하다고 판단되는 것들에 대해 상관관계를 알아볼 예정
# CODE_GENDER과 AMT_GOODS_PRICE, AMT_CREDIT, AMT_ANNUITY, AMT_INCOME_TOTAL

amt_gender = app_train[['TARGET','CODE_GENDER', 'AMT_GOODS_PRICE','AMT_CREDIT','AMT_ANNUITY','AMT_INCOME_TOTAL']]
amt_gender_corrs = amt_gender.corr()
amt_gender_corrs


In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(amt_gender_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

#성별에 따른 AMT_GOODS_PRICE, AMT_CREDIT, AMT_ANNUITY, AMT_INCOME_TOTAL과의 연관성이 있지않을까? 생각했는데
# AMT_GOODS_PRICE, AMT_CREDIT, AMT_ANNUITY 끼리의 연관만 높은걸로 알 수 있었다. 대출금액에 대한 컬럽이니 상관관계가 높은것은 당연하다
# 이 3가지에 대한 Feature 가공 필요

In [ ]:
#CNT_CHILDREN이 0 일경우, NAME_FAMILY_STATUS결혼을 했을 경우 상환을 더 잘한다는 것을 볼 수 있었다.
amt_gender = app_train[['TARGET','CNT_CHILDREN', 'NAME_FAMILY_STATUS','AMT_GOODS_PRICE','AMT_CREDIT','AMT_ANNUITY','AMT_INCOME_TOTAL']]
amt_gender_corrs = amt_gender.corr()
amt_gender_corrs

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(amt_gender_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');
# 큰 상관관계는 없다.

In [ ]:
#나이가 어릴수록 대출상환을 더 못한다는 EDA결과가 있었다.
amt_gender = app_train[['TARGET','DAYS_BIRTH', 'AMT_GOODS_PRICE','AMT_CREDIT','AMT_ANNUITY','AMT_INCOME_TOTAL']]
amt_gender_corrs = amt_gender.corr()
amt_gender_corrs


In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(amt_gender_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

In [ ]:
#DAYS_EMPLOYED 직업이 있을경우의 상관관계
amt_gender = app_train[['TARGET','DAYS_EMPLOYED', 'AMT_GOODS_PRICE','AMT_CREDIT','AMT_ANNUITY','AMT_INCOME_TOTAL']]
amt_gender_corrs = amt_gender.corr()
amt_gender_corrs

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(amt_gender_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');
#큰 유의미한 feature 상관관계는 없는듯하다

In [ ]:
# 거주지역에 따른 상관관계
amt_gender = app_train[['TARGET','REGION_RATING_CLIENT', 'AMT_GOODS_PRICE','AMT_CREDIT','AMT_ANNUITY','AMT_INCOME_TOTAL']]
amt_gender_corrs = amt_gender.corr()
amt_gender_corrs

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(amt_gender_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');
#별 유의미한 정보 없음

In [ ]:
# 고객 직장유형에 따른 상관관계
amt_gender = app_train[['TARGET','ORGANIZATION_TYPE', 'AMT_GOODS_PRICE','AMT_CREDIT','AMT_ANNUITY','AMT_INCOME_TOTAL']]
amt_gender_corrs = amt_gender.corr()
amt_gender_corrs

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(amt_gender_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');
#별 유의미한 정보 없음

In [ ]:
#Feature 
# 나이와 대출 금액   DAYS_BIRTH / AMT_CREDIT
# 고객 직장 유형과  대출금액  ORGANIZATION_TYPE / AMT_CREDIT
# 고객주택 유형과 대출금액 NAME_HOUSING_TYPE /AMT_CREDIT
# 대출 신청전 현 직업 유지기간과 대출 금액 DAYS_EMPLOYED /AMT_CREDIT
#소득유형에 따른/.. NAME_INCOME_TYPE /  AMT_CREDIT

apps['APPS_BIRTH_CREDIT_RATIO'] = apps['DAYS_BIRTH'] / apps['AMT_CREDIT']
apps['APPS_ORAGANIZATION_CREDIT_RATIO'] = apps['ORGANIZATION_TYPE'] / apps['AMT_CREDIT']
apps['APPS_HOUSING_CREDIT_RATIO'] = apps['NAME_HOUSING_TYPE'] / apps['AMT_CREDIT']
apps['APPS_EMPLOYED_CREDIT_RATIO'] = apps['DAYS_EMPLOYED'] / apps['AMT_CREDIT']
apps['APPS_INCOME_CREDIT_RATIO'] = apps['NAME_INCOME_TYPE'] / apps['AMT_CREDIT']

In [ ]:
#Feature 
# 나이와 대출 금액   DAYS_BIRTH / AMT_ANNUITY
# 고객 직장 유형과  대출금액  ORGANIZATION_TYPE / AMT_ANNUITY
# 고객주택 유형과 대출금액 NAME_HOUSING_TYPE /AMT_ANNUITY
# 대출 신청전 현 직업 유지기간과 대출 금액 DAYS_EMPLOYED /AMT_ANNUITY
#소득유형에 따른/.. NAME_INCOME_TYPE /  AMT_ANNUITY

apps['APPS_BIRTH_ANNUITY_RATIO'] = apps['DAYS_BIRTH'] / apps['AMT_ANNUITY']
apps['APPS_ORAGANIZATION_ANNUITY_RATIO'] = apps['ORGANIZATION_TYPE'] / apps['AMT_ANNUITY']
apps['APPS_HOUSING_ANNUITY_RATIO'] = apps['NAME_HOUSING_TYPE'] / apps['AMT_ANNUITY']
apps['APPS_EMPLOYED_ANNUITY_RATIO'] = apps['DAYS_EMPLOYED'] / apps['AMT_ANNUITY']
apps['APPS_INCOME_ANNUITY_RATIO'] = apps['NAME_INCOME_TYPE'] / apps['AMT_ANNUITY']

In [ ]:
apps.shape

<모델링>
- LGBM 기반 학습
- 모델링

In [ ]:
# #해당 조건문이면 list화
# object_columns = apps.dtypes[apps.dtypes == 'object'].index.tolist() 

# for column in object_columns:
#     apps[column] = pd.factorize(apps[column])[0]
    
# # apps.info() #object 타입이 없어졌다!

In [ ]:
#apps 데이터를 다시 분리
app_train = apps[-apps['TARGET'].isnull()]  # NULL아닌 것들은 train으로
app_test = apps[apps['TARGET'].isnull()] # NULL 처리 된것 --> 나중에 TARGET column 삭제
app_train.shape, app_test.shape

In [ ]:
# # app_test의 TARGET drop
# app_test = app_test.drop('TARGET', axis=1, inplace=False)
# app_test.shape
# #원래대로 121개의 열

In [ ]:
#학습데이터를 검증 데이터로 분히라고 LGBM Classifier로 학습수행
ftr_app = app_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = app_train['TARGET']

In [ ]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
eval_metric= 'auc', verbose= 100, early_stopping_rounds= 50)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

#위에서 랜덤으로 시각화해봤더니 유의미한 결과가 나왔던 DAYS_BIRTH 부분이 높은 Feature Importance가 나왔다.

In [ ]:
# 높은 Featue importance 순위를 가진 column에 대해 시각화를 해보고 중요한 column을 확인하기
columns = ['AMT_INCOME_TOTAL','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
           'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'EXT_SOURCE_1', 
           'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 
           'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']
show_hist_by_target(app_train, columns)

Kaggle Submit
- Private score, Public Score 확인

In [ ]:
preds = clf.predict_proba(apps_test.drop(['SK_ID_CURR','TARGET'], axis=1))[:,1]